In [1]:
HF_USR_NAME = 'shirwu'
TOOL_QA_ROOT = '/dfs/project/kgrlm/shirwu/msr_intern/home/t-yingxinwu/msr_intern/ToolQA-rebuttal'

### Upload to Huggingface

In [2]:
import pandas as pd
from datasets import Dataset
from datasets import DatasetDict

level = 'hard'
dataset = 'scirex'

dataset_dir = f'{dataset}-{level}.jsonl'
hf_dataset_name = f'toolqa_{dataset}_{level}'

df = pd.read_json(dataset_dir, lines=True)
df.head()

df['answer'] = df['answer'].apply(lambda x: str(x))
dataset = Dataset.from_pandas(df)

In [3]:
dataset_dict = DatasetDict({'train': dataset})
# push to hf for the ease for using dspy
# dataset_dict.push_to_hub(repo_id=hf_dataset_name, private=True)

## Setting Up

* ToolQA

Before loading our datasets and going to the execution part, we'll need to configure the `lm` in `dspy.settings`. For the purpose of this notebook we'll be using `gpt-4o`.

In [4]:
import os
import dspy
import warnings
warnings.simplefilter("ignore", UserWarning)
warnings.simplefilter("ignore", FutureWarning) 


dspy.settings.configure(
    lm=dspy.OpenAI(
        model="gpt-4o",
        api_key=os.getenv("OPENAI_API_KEY"),
        max_tokens=4000,
        temperature=0
    )
)

## Defining Signature

In [5]:
class ToolQASignature(dspy.Signature):
    """You will be given a question. Your task is to answer the question with a short response. 
    """
    
    question: str = dspy.InputField(
        prefix="Question:",
        desc="question to ask",
        format=lambda x: x.strip(),
    )
    answer: str = dspy.OutputField(
        prefix="Answer:",
        desc="answer to the question",
    )


## Loading Datasets

In [6]:
from random import sample
from dspy.datasets import DataLoader

dl = DataLoader()

In [7]:
tool_qa = dl.from_huggingface(
    f'{HF_USR_NAME}/' + hf_dataset_name,
    split="train",
    input_keys=("question", "answer"),
)

In [8]:
len(tool_qa)

100

In [9]:
import random
# set seed
random.seed(42)

train_idx = random.sample(range(len(tool_qa)), 40)
remaining_idx = list(set(range(len(tool_qa))) - set(train_idx))
test_idx = random.sample(remaining_idx, 60)

toolqa_train = [
    dspy.Example(question=example.question, answer=example.answer).with_inputs("question", "paper_id")
    for example in [tool_qa[i] for i in train_idx]
]
toolqa_test = [
    dspy.Example(question=example.question, answer=example.answer).with_inputs("question", "paper_id")
    for example in [tool_qa[i] for i in test_idx]
]

## Setting Up Tools

We'll setup `Avatar` modules for both signatures and all the `tools` can be used by each of the dataset. `Tool` is a pydantic model that Avatar expects the `tools` to be composed as more specifically it have 4 fields:

* `name` : Name of the tool
* `input_type` : Type of input the tool accepts
* `output_type` : Type of output the tool returns
* `tool` : The actual tool object

In [10]:
import os
import time
import uuid
import numpy as np
import jsonlines
from concurrent.futures import ProcessPoolExecutor
import sentence_transformers
import chromadb
from os import path as osp
from chromadb.config import Settings

EMBED_MODEL_NAME = "sentence-transformers/all-mpnet-base-v2"
CHROMA_PERSIST_DIRECTORY = osp.join(TOOL_QA_ROOT, "data/chroma_db/scirex-v2")
CHROMA_COLLECTION_NAME = "all"
CHROMA_SERVER_HOST = "localhost"
CHROMA_SERVER_HTTP_PORT = "8000"
FILE_PATH = osp.join(TOOL_QA_ROOT, "data/external_corpus/scirex/Preprocessed_Scirex.jsonl")

def sentence_embedding(model, texts):
    embeddings = model.encode(texts)
    return embeddings

def create_chroma_db(chroma_server_host, chroma_server_http_port, collection_name):
    chroma_client = chromadb.Client(Settings(
        chroma_api_impl="rest",
        chroma_server_host=chroma_server_host,
        chroma_server_http_port=chroma_server_http_port,
    ))
    collection = chroma_client.get_or_create_collection(name=collection_name)
    return collection

def create_chroma_db_local(persist_directory, collection_name):
    chroma_client = chromadb.PersistentClient(path=persist_directory)
    collection = chroma_client.get_or_create_collection(name=collection_name)
    return collection

def insert_to_db(texts, model_name, cuda_idx, db):
    # use cpu
    model = sentence_transformers.SentenceTransformer(model_name, device='cpu')
    # model = sentence_transformers.SentenceTransformer(model_name, device=f"cuda:{cuda_idx}")

    batch_embeddings = []
    batch_texts = []
    start_time = time.time()
    print(f"Total Articles to process: {len(texts)}, Current Thread: {cuda_idx}.")
    for i, text in enumerate(texts):
        # 2. generate embedding
        embeddings = sentence_embedding(model, text).tolist()

        batch_embeddings.append(embeddings)
        batch_texts.append(text)
        # 3. add to vectorstore per 500 articles or last article
        if i % 100 == 0 or i == len(texts)-1:
            batch_ids = [str(uuid.uuid1()) for _ in batch_texts]
            db.add(
                embeddings=batch_embeddings,
                documents=batch_texts,
                ids = batch_ids
            )
            batch_embeddings = []
            batch_texts = []
            print(f"Completed Processing article count: {i}, Current Thread: {cuda_idx}, Time took: {time.time() - start_time}.")
    print(f"Thread {cuda_idx} Completed. Total time took for thread: {time.time() - start_time}.")


# Multi-processing
def query_llm(query, is_local=True, start=None, end=None):
    cuda_idxes = [0]
    number_of_processes = len(cuda_idxes)
    input_texts = []
    db = create_chroma_db_local(CHROMA_PERSIST_DIRECTORY, CHROMA_COLLECTION_NAME)
    with open(FILE_PATH, 'r') as f:
        for item in jsonlines.Reader(f):
            input_texts.append(item["content"])
    # input_texts = np.array_split(input_texts, number_of_processes)

    args = ((input_texts[i], EMBED_MODEL_NAME, cuda_idxes[i], is_local) for i in range(number_of_processes))

    # if there is no file under the directory "/localscratch/yzhuang43/ra-llm/retrieval_benchmark/data/chroma_db/agenda", insert the data into the db
    # You should run insert_to_db the first time!
    if len(os.listdir(CHROMA_PERSIST_DIRECTORY)) == 0:
        insert_to_db(input_texts, model_name=EMBED_MODEL_NAME, cuda_idx=0, db=db)

    input_paths = np.array_split(input_texts, number_of_processes)
    with ProcessPoolExecutor(number_of_processes) as executor:
        executor.map(insert_to_db, args)
    # use cpu
    model = sentence_transformers.SentenceTransformer(EMBED_MODEL_NAME, device='cpu')
    # model = sentence_transformers.SentenceTransformer(EMBED_MODEL_NAME, device=f"cuda:0")
    query_embedding = sentence_embedding(model, query).tolist()
    results = db.query(query_embeddings=query_embedding, n_results=3)
    retrieval_content = [result for result in results['documents'][0]]
    # print(retrieval_content)
    retrieval_content = '\n'.join(retrieval_content)
    return retrieval_content

query = "What is an atom"
print(query_llm(query))

paragraph : Sentence Level For representing a document , one can split it up into sentences , with each memory slot encoding one sentence . Both the key and the value encode the entire sentence as a bag - of - words . As the key and value are the same in this case , this is identical to a standard MemNN and this approach has been used in several papers .
paragraph : Window Level Documents are split up into windows of words ; in our tasks we only include windows where the center word is an entity . Windows are represented using bag - of - words . Window representations for MemNNs have been shown to work well previously . However , in Key - Value MemNNs we encode the key as the entire window , and the value as only the center word , which is not possible in the MemNN architecture . This makes sense because the entire window is more likely to be pertinent as a match for the question ( as the key ) , whereas the entity at the center is more pertinent as a match for the answer ( as the valu

In [11]:
from dspy.predict.avatar import Tool, Avatar
from langchain_community.utilities import GoogleSerperAPIWrapper, ArxivAPIWrapper, WikipediaAPIWrapper
from langchain.tools import BaseTool, StructuredTool, tool

def RETRIEVE(query: str) -> str:
    """If you want to search for some paper information, you can use this tool and input a natural language query. For example, RETRIEVE(\'Which method achieves the highest PCK score?\') returns relevant paper paragraph and meta data."""
    return query_llm(query)

tools = [
    Tool(
        tool=StructuredTool.from_function(RETRIEVE),
        name="RETRIEVE",
        desc="If you want to search for some paper information, you can use this tool and input a natural language query. For example, RETRIEVE('Which method achieves the highest PCK score?') returns relevant paper paragraph and meta data."
    ),
    Tool(
        tool=GoogleSerperAPIWrapper(),
        name="WEB_SEARCH",
        desc="If you have a question, you can use this tool to search the web for the answer."
    ),
    Tool(
        tool=ArxivAPIWrapper(),
        name="ARXIV_SEARCH",
        desc="Pass the arxiv paper id to get the paper information.",
        input_type="Arxiv Paper ID",
    )
]

Once we have defined our `tools`, we can now create an `Avatar` object by passing the `tools` and `signature`. It takes 2 more optional parameters `verbose` and `max_iters`. `verbose` is used to display the logs and `max_iters` is used to control the number of iterations in multi step execution. 

An avatar agent stops the tool usage iteration once it reaches `max_iters` or when it prompts `Finish`. You can also create custom tools too, all you need to make sure is:

* You pass is a class object.
* Implements `__init__` and `run` method.
* Must take 1 string a input and returns 1 string as output.

If your tool doesn't return or takes input a string then you can make a custom wrapper to take care of that for now. In future we'll try to enable a diverse tool usage.

In [12]:
actor_agent = Avatar(
    tools=tools,
    signature=ToolQASignature,
    verbose=False,
    max_iters=10
)

In [13]:
import time
import logging
from dataclasses import dataclass, field
from typing import Dict, List, Optional
from datetime import datetime
import tiktoken
from concurrent.futures import ThreadPoolExecutor, as_completed
import warnings
import copy
import tqdm
import logging
import warnings
import os

# Set up logging
# logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)


# Disable all INFO logging
logging.getLogger().setLevel(logging.WARNING)

# Silence all loggers that might be chatty
loggers_to_silence = [
    "httpx",
    "httpcore",
    "openai",
    "arxiv",
    "dspy",
    "langchain",
    "langchain_community",
    "requests",
    "urllib3",
    "tiktoken",
    "asyncio",
    "faiss",
    "anthropic"
]

for logger_name in loggers_to_silence:
    logging.getLogger(logger_name).setLevel(logging.WARNING)

# Suppress specific warnings
warnings.filterwarnings('ignore', category=UserWarning)
warnings.filterwarnings('ignore', category=DeprecationWarning)
warnings.filterwarnings('ignore', category=FutureWarning)

os.environ['TOKENIZERS_PARALLELISM'] = 'false'  # Disable tokenizer parallelism warning

## Evaluation

Open enden QA tasks are hard to evaluate on rigid metrics like exact match. So, we'll be using an improvised LLM as Judge for the evaluation of our model on test set.

In [14]:
class Evaluator(dspy.Signature):
    """Please act as an impartial judge to evaluate whether the answer is correct based on the ground truth answer"""
    
    question: str = dspy.InputField(
        prefix="Question:",
        desc="question to ask",
    )
    reference_answer: str = dspy.InputField(
        prefix="Ground Truth Answer:",
        desc="Ground truth answer to the question.",
    )
    answer: str = dspy.InputField(
        prefix="Answer:",
        desc="Answer to the question given by the model.",
    )
    rationale: str = dspy.OutputField(
        prefix="Rationale:",
        desc="Explanation of why the answer is correct or incorrect.",
    )
    is_correct: float = dspy.OutputField(
        prefix="Correct:",
        desc="Whether the answer is correct. Give 0 if incorrect, 1 if correct, (0, 1) if partially correct.",
    )


evaluator = dspy.TypedPredictor(Evaluator)


def metric(example, prediction, trace=None):  
    # We found sometimes the ground truth answers are incomplete or the answer
    # is part of the ground truth answer. Therefore, for better comparison, 
    # we use a continuous value for the correct score   
    acc = float(
        evaluator(
            question=example.question,
            answer=prediction.answer,
            reference_answer=example.answer
        ).is_correct
    ) 
    print(prediction.answer, '|', example.answer, '=>', acc)
    return acc

print(toolqa_train[0])
metric(toolqa_train[0], prediction=dspy.Example(answer='physics'))

Example({'question': 'Which method achieves the highest PCK score on Leeds_Sports_Poses dataset for Pose_Estimation task?', 'answer': 'Pyramid_Residual_Modules__PRMs_'}) (input_keys={'paper_id', 'question'})
physics | Pyramid_Residual_Modules__PRMs_ => 0.0


0.0

For evaluation we can't use `dspy.Evaluate`, reason being that `Avatar` changes it's signature per iteration by adding the actions and it's results to it as fields. So we can create our own hacky thread safe evaluator for it.

In [15]:
import time
import logging
from dataclasses import dataclass, field
from typing import Dict, List, Optional
from datetime import datetime
import tiktoken
from concurrent.futures import ThreadPoolExecutor, as_completed
import warnings
import copy

# Set up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

@dataclass
class APICallMetrics:
    timestamp: datetime
    tool_name: str
    tokens_in: int = 0
    tokens_out: int = 0
    execution_time: float = 0.0

@dataclass
class AvatarMetrics:
    total_calls: int = 0
    total_tokens_in: int = 0
    total_tokens_out: int = 0
    total_execution_time: float = 0.0
    calls_by_tool: Dict[str, int] = field(default_factory=dict)
    api_call_history: List[APICallMetrics] = field(default_factory=list)
    
    def add_call(self, metrics: APICallMetrics):
        self.total_calls += 1
        self.total_tokens_in += metrics.tokens_in
        self.total_tokens_out += metrics.tokens_out
        self.total_execution_time += metrics.execution_time
        self.calls_by_tool[metrics.tool_name] = self.calls_by_tool.get(metrics.tool_name, 0) + 1
        self.api_call_history.append(metrics)
    
    def merge(self, other: 'AvatarMetrics'):
        """Merge another AvatarMetrics instance into this one"""
        self.total_calls += other.total_calls
        self.total_tokens_in += other.total_tokens_in
        self.total_tokens_out += other.total_tokens_out
        self.total_execution_time += other.total_execution_time
        for tool, count in other.calls_by_tool.items():
            self.calls_by_tool[tool] = self.calls_by_tool.get(tool, 0) + count
        self.api_call_history.extend(other.api_call_history)

    def estimate_cost(self, model_name: str = "gpt-4") -> float:
        pricing = {
            "gpt-4": {"input": 2.5, "output": 10.0},
        }
        if model_name not in pricing:
            raise ValueError(f"Unknown model: {model_name}")
        
        rates = pricing[model_name]
        input_cost = (self.total_tokens_in / 1000000) * rates["input"]
        output_cost = (self.total_tokens_out / 1000000) * rates["output"]
        return input_cost + output_cost

class AvatarWithMetrics(Avatar):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.metrics = AvatarMetrics()
        self.tokenizer = tiktoken.encoding_for_model("gpt-4")
    
    def _count_tokens(self, text: str) -> int:
        try:
            return len(self.tokenizer.encode(str(text)))
        except Exception as e:
            logger.warning(f"Error counting tokens: {e}")
            return 0

    def _wrapped_tool_call(self, tool, input_text: str) -> str:
        start_time = time.time()
        tokens_in = self._count_tokens(input_text)
        
        try:
            result = tool.run(input_text)
        except Exception as e:
            logger.error(f"Tool execution error: {e}")
            raise
        finally:
            execution_time = time.time() - start_time
            tokens_out = self._count_tokens(str(result))
            
            metrics = APICallMetrics(
                timestamp=datetime.now(),
                tool_name=tool.name,
                tokens_in=tokens_in,
                tokens_out=tokens_out,
                execution_time=execution_time
            )
            self.metrics.add_call(metrics)
            
        return result

    def __call__(self, *args, **kwargs):
        start_time = time.time()
        result = super().__call__(*args, **kwargs)
        total_time = time.time() - start_time
        
        metrics = APICallMetrics(
            timestamp=datetime.now(),
            tool_name="main_llm",
            tokens_in=self._count_tokens(str(args) + str(kwargs)),
            tokens_out=self._count_tokens(str(result)),
            execution_time=total_time
        )
        self.metrics.add_call(metrics)
        
        return result

def multi_thread_executor(test_set, signature, num_threads=60):
    total_score = 0
    total_examples = len(test_set)
    combined_metrics = AvatarMetrics()

    start_time = time.time()
    with ThreadPoolExecutor(max_workers=num_threads) as executor:
        futures = []
        for example in test_set:
            def process_with_metrics(example=example):
                try:
                    avatar = AvatarWithMetrics(signature, tools=tools, verbose=False, max_iters=10)
                    prediction = avatar(**example.inputs().toDict())
                    return metric(example, prediction), avatar.metrics
                except Exception as e:
                    print(e)
                    return 0, AvatarMetrics()

            futures.append(executor.submit(process_with_metrics))

        for future in tqdm.tqdm(futures, total=total_examples, desc="Processing examples"):
            score, metrics = future.result()
            total_score += score
            # Only combine token counts and call counts, not execution times
            combined_metrics.total_calls += metrics.total_calls
            combined_metrics.total_tokens_in += metrics.total_tokens_in
            combined_metrics.total_tokens_out += metrics.total_tokens_out
            for tool, count in metrics.calls_by_tool.items():
                combined_metrics.calls_by_tool[tool] = combined_metrics.calls_by_tool.get(tool, 0) + count
            combined_metrics.api_call_history.extend(metrics.api_call_history)
    
    total_execution_time = time.time() - start_time
    combined_metrics.total_execution_time = total_execution_time

    avg_metric = total_score / total_examples
    return avg_metric, combined_metrics

def single_thread_executor(test_set, signature):
    total_score = 0
    total_examples = len(test_set)
    combined_metrics = AvatarMetrics()

    for example in tqdm.tqdm(test_set, desc="Processing examples"):
        try:
            avatar = AvatarWithMetrics(signature, tools=tools, verbose=False, max_iters=10)
            prediction = avatar(**example.inputs().toDict())
            score = metric(example, prediction)
            total_score += score
            # Combine metrics from this run
            for call in avatar.metrics.api_call_history:
                combined_metrics.add_call(call)
        except Exception as e:
            print(e)

    avg_metric = total_score / total_examples
    return avg_metric, combined_metrics

def format_metrics_report(metrics: AvatarMetrics, model_name: str = "gpt-4") -> str:
    cost = metrics.estimate_cost(model_name)
    
    report = f"""
Avatar Execution Metrics Report
==============================
Execution Time: {metrics.total_execution_time:.2f} seconds
Total API Calls: {metrics.total_calls}
Total Tokens: {metrics.total_tokens_in + metrics.total_tokens_out:,} ({metrics.total_tokens_in:,} in, {metrics.total_tokens_out:,} out)
Estimated Cost: ${cost:.4f}

Average Time per Call: {metrics.total_execution_time / metrics.total_calls:.2f} seconds

Tool Usage Breakdown:
-------------------
"""
    for tool, count in sorted(metrics.calls_by_tool.items()):
        report += f"{tool}: {count} calls\n"

    return report

## One-shot result

In [16]:
score, metrics = multi_thread_executor(toolqa_test, ToolQASignature)

Processing examples:   2%|▏         | 1/60 [00:12<12:01, 12.22s/it]

The method that achieves the highest Score score on the Atari_2600_Name_This_Game dataset for the Atari_Games task is MuZero. | IQN => 0.0
EffNet-L2 (SAM) achieves the highest Percentage_correct score of 96.08 on the CIFAR-100 dataset for the Image Classification task. | Res2NeXt-29 => 0.0
The highest reported Mean_IoU score on the CamVid dataset for Semantic Segmentation is 66.1%. | PSPNet => 0.0
The method that achieves the highest F1 score on the CoNLL 2003 English dataset for Named Entity Recognition (NER) is ACE + document-context with an F1 score of 94.6. | CVT___Multi-Task => 0.0
The VGG_Resnet_LACE_BiLSTM_acoustic_model trained on SWB Fisher CH is evaluated on the Switchboard and Hub500 datasets for the Speech Recognition task. | swb_hub_500_WER_fullSWBCH => 0.5
The DQN_hs method is evaluated on the Atari 2600 games, as mentioned in the context of various research papers discussing reinforcement learning methods applied to Atari games. | Atari_2600_Chopper_Command => 0.0
The Di

Processing examples: 100%|██████████| 60/60 [02:05<00:00,  2.09s/it] 

The PNN method is evaluated on the Bing_News dataset for Click-Through Rate Prediction using metrics such as Area Under the Curve (AUC). | AUC, Log_Loss => 0.5


In [17]:
# print(f"Average Score on ArxivQA before opitmization: {aqa_score:.2f}")
print(f"Test Score: {score:.2f}")
print(format_metrics_report(metrics))

Test Score: 0.24

Avatar Execution Metrics Report
Execution Time: 127.10 seconds
Total API Calls: 60
Total Tokens: 91,949 (1,702 in, 90,247 out)
Estimated Cost: $0.9067

Average Time per Call: 2.12 seconds

Tool Usage Breakdown:
-------------------
main_llm: 60 calls



In [18]:
# Pure batch sampling

from new_optimizer import AvatarOptimizerWithMetrics

batch_sampling_monkey = AvatarOptimizerWithMetrics(
    metric=metric,
    max_iters=0,
    max_negative_inputs=10,
    max_positive_inputs=10,
    lower_bound=0.5,
    upper_bound=0.5
)
result = batch_sampling_monkey.compile(
    student=actor_agent,
    trainset=toolqa_train
)
batch_num = 2
batch_sampling_monkey.thread_safe_evaluator_batch(toolqa_test, result['agent'], batch_num)


                Optimization Process Metrics
                Total Execution Time: 0.00 seconds
                Evaluation Time: 0.00 seconds
                Total API Calls: 0
                - Comparator calls: 0
                - Feedback instruction calls: 0

                Token Usage:
                ----------
                Total Tokens: 0
                - Input tokens: 0
                - Output tokens: 0

                Cost Analysis:
                ------------
                Estimated Total Cost: $0.0000
                
Processing batch 1 of 2...
The method that achieves the highest Score score on the Atari_2600_Name_This_Game dataset for the Atari_Games task is MuZero. | IQN => 0.0
EffNet-L2 (SAM) achieves the highest Percentage_correct score of 96.08 on the CIFAR-100 dataset for the Image Classification task. | Res2NeXt-29 => 0.0
The method that achieves the highest F1 score on the CoNLL 2003 English dataset for Named Entity Recognition (NER) is ACE + document-con

Processing examples:   0%|          | 0/60 [00:00<?, ?it/s]

The method that achieves the highest MAP score on the WikiQA dataset for the Question Answering task is TANDA, which achieved a MAP score of 92%. | Key-Value_Memory_Network => 0.0
The U-Net method for Skin Cancer Segmentation is evaluated on several datasets, including the ISIC-2018 dataset and the HAM10000 dataset. | Kaggle_Skin_Lesion_Segmentation => 0.0
LiteFlowNet achieves the highest Average End-Point Error score for Optical Flow Estimation on the Sintel final pass and KITTI benchmarks. | Sintel-final => 0.5
The highest Number_of_params score on the WikiText-2 dataset for the Language Modelling task is achieved by the method described in the paper "Direct Output Connection for a High-Rank Language Model" by Sho Takase, Jun Suzuki, and Masaaki Nagata. This method improves the current state-of-the-art language model and achieves the best score on the WikiText-2 dataset. | AWD-LSTM-DOC => 0.0
The search did not yield specific datasets for the Deep_Speech method evaluation in Speech R

Processing examples:   3%|▎         | 2/60 [01:25<41:21, 42.79s/it]

The PNN method for Click-Through Rate Prediction on the Bing_News dataset is evaluated using metrics such as the Area Under the Curve (AUC). | AUC, Log_Loss => 0.5


...

The PNN method is evaluated on the Bing_News dataset for Click-Through Rate Prediction using metrics such as the Area Under the Curve (AUC). | AUC, Log_Loss => 0.5


Processing examples: 100%|██████████| 60/60 [01:53<00:00,  1.89s/it]

The NICE method evaluation metrics on the CIFAR-10 dataset for the Image Generation task are not explicitly mentioned in the retrieved documents. However, common metrics for evaluating image generation tasks include Inception Score and Fréchet Inception Distance (FID). | NLL_Test => 0.0



Batch Evaluation Metrics Report
Total Execution Time: 247.90 seconds
Average Time per Batch: 123.95 seconds
Best Score: 0.258 (Batch 1)
Total Tokens: 192,239 (3,284 in, 188,955 out)
Total Cost: $11.4358

Per-Batch Performance:
--------------------

Batch 1:
  Score: 0.258
  Execution Time: 124.19s
  Tokens: 97,684 (1,642 in, 96,042 out)
  Cost: $5.8118

Batch 2:
  Score: 0.250
  Execution Time: 123.71s
  Tokens: 94,555 (1,642 in, 92,913 out)
  Cost: $5.6240


(0.25833333333333336,
 {'batches': [{'batch_id': 1,
    'score': 0.25833333333333336,
    'execution_time': 124.19007873535156,
    'tokens_in': 1642,
    'tokens_out': 96042,
    'cost': 5.81178},
   {'batch_id': 2,
    'score': 0.25,
    'execution_time': 123.70830774307251,
    'tokens_in': 1642,
    'tokens_out': 92913,
    'cost': 5.62404}],
  'total_execution_time': 247.89838647842407,
  'total_tokens_in': 3284,
  'total_tokens_out': 188955,
  'best_batch': 1,
  'total_cost': 11.43582,
  'final_score': 0.25833333333333336,
  'average_batch_time': 123.94919323921204})

## Optimization

For the optimization of the `Actor` we'll be using `AvatarOptimizer`. It's a DSPy implementation of the [Avatar](https://github.com/zou-group/avatar/) method that optimizes the `Actor` for the given `tools` using a comparator module that optimizes Actor instruction. Note, that Actor is the Module that directs tool execution and flow, it's not the signature that we are passing. It doesn't optimize the instruction of the signature we pass. It takes the following parameters:

* `metric`: Metric that we'll be optimizing for
* `max_iters`: Maximum number of iterations for the optimizer
* `lower_bound`: Lower bound for the metric to classify example as negative
* `upper_bound`: Upper bound for the metric to classify example as positive
* `max_positive_inputs`: Maximum number of positive inputs sampled for comparator
* `max_negative_inputs`: Maximum number of negative inputs sampled for comparator
* `optimize_for`: Whether we want to maximize the metric or minimize it during optimization

Once the optimizer is done we can get the optimized actor and use it for the evaluation.

In [19]:
from new_optimizer import AvatarOptimizerWithMetrics

iterative_monkey = AvatarOptimizerWithMetrics(
    metric=metric,
    max_iters=1,
    max_negative_inputs=10,
    max_positive_inputs=10,
    lower_bound=0.5,
    upper_bound=0.5
)

In [20]:
result = iterative_monkey.compile(
    student=actor_agent,
    trainset=toolqa_train
)

OmniPose achieves the highest PCK score of 99.5% on the Leeds Sports Poses dataset for the Pose Estimation task. | Pyramid_Residual_Modules__PRMs_ => 0.0


Processing examples:   0%|          | 0/40 [00:00<?, ?it/s]

The method achieving the highest MRR score on the FB15k dataset for the Link Prediction task is AutoKGE with an MRR of 0.861. | TuckER => 0.0
The big transformer model achieved the highest BLEU score of 28.4 on the WMT2014 English-German dataset for the Machine Translation task. | Weighted_Transformer__large_ => 0.0
The ByteNet method is evaluated on the English-to-German WMT translation task for Machine Translation. | WMT2014_English-French => 0.0
The method that achieves the highest SSIM score on the Vid4 - 4x upscaling dataset for Video Super-Resolution is EvTexture+ with an SSIM score of 0.8983. | VESPCN => 0.0
The method that achieves the highest Medium_Human-Normalized_Score on the Atari-57 dataset for Atari Games is LBC with a score of 10077.52%. | Ape-X => 0.0
The Bi-LSTM trained on FCE method achieves the highest F0.5 score on the FCE dataset for the Grammatical Error Detection task, as indicated by the result from the paper by Masahiro Kaneko and Mamoru Komachi. | CoNLL-2014_

Processing examples:   5%|▌         | 2/40 [00:56<17:58, 28.37s/it]

The Duel_noop method is evaluated on 57 Atari games for the Atari_Games task. | Atari_2600_Time_Pilot => 0.0
The DeepFM method achieves the highest Log_Loss score for the Click-Through Rate Prediction task on the Criteo dataset. The Criteo dataset is a well-known ad tech industry benchmarking dataset used for evaluating CTR prediction models. | Criteo => 1.0
The TARNet method is evaluated on the semi-synthetic IHDP dataset and the Jobs dataset, which includes both a randomized and a non-randomized component, for the Causal Inference task. | IDHP => 0.5
The CNN___Bi-RNN___CTC__speech_to_letters___25_9__WER_if_trainedonlyon_SWB method is evaluated on the swb_hub_500_WER_fullSWBCH dataset using the Word Error Rate (WER) metric. | Percentage_error => 1.0
CornerNet-Squeeze is evaluated on the COCO dataset for the Real-Time_Object_Detection task. | COCO => 1.0
The PSENet-1s method is evaluated on the SCUT-CTW1500 dataset using precision, recall, and F-measure metrics. | F-Measure => 0.5
The 

...

Processing examples:  28%|██▊       | 11/40 [01:04<01:35,  3.28s/it]

The LapSRN method is evaluated on the Urban100 - 4x upscaling dataset using two evaluation metrics: peak signal-to-noise ratio (PSNR) and structural similarity index (SSIM). | PSNR => 0.5
The MTGAE method is evaluated on the Pubmed dataset for the Link_Prediction task using common evaluation metrics for link prediction tasks, such as AUC (Area Under the Curve) and possibly other metrics like precision, recall, or F1-score, although specific metrics for MTGAE were not found in the retrieved documents. | Accuracy => 0.0
The MT-DNN method is evaluated on the MultiNLI dataset using accuracy as the primary metric for the Natural Language Inference task. | Matched, Mismatched => 0.0


Processing examples: 100%|██████████| 40/40 [01:08<00:00,  1.72s/it]

The Subgraph_embeddings method evaluation metrics on the WebQuestions dataset for the Question_Answering task are not explicitly found in the available resources. It is likely that standard metrics such as accuracy, precision, recall, and F1-score are used, but specific details would require access to the original research paper or documentation on the Subgraph_embeddings method. | F1 => 0.0



Evaluation Metrics Report
Execution Time: 75.46 seconds
Total Tokens: 54,656 (1,090 in, 53,566 out)
Total Cost: $0.5384
Average Score: 0.263
Average Score: 0.2625
Evaluation Cost: $0.5384
Generated new instruction: New Instruction: You will be given `Tools`, which is a list of resources to use to accomplish the `Goal`. When presented with a user query, your task is to decide which tool to use and what input values to provide. To enhance the effectiveness of your actions, begin by formulating specific and detailed queries. Break down complex queries into simpler components to improve the relevance and accuracy of the results. This will help in identifying the most appropriate tools to use for each part of the query.

For each query, implement a strategy to combine multiple tools, especially for complex or broad questions. Start with a web search to gather initial information, then use retrieval or arXiv search for more detailed insights. This combination will allow you to gather compre

In [21]:
optimized_actor_agent = result["agent"]
optimization_metrics = result["metrics"]

# Now you can process the metrics
print(f"Total optimization cost: ${optimization_metrics['total_cost']:.4f}")
print(f"Final score achieved: {optimization_metrics['final_score']:.3f}")

# Analyze per-iteration performance
for iteration in optimization_metrics['iteration_details']:
    print(f"\nIteration {iteration['iteration']}:")
    print(f"Score: {iteration['score']:.3f}")
    print(f"Comparator tokens in: {iteration['comparator_metrics']['tokens_in']}")
    print(f"Comparator tokens out: {iteration['comparator_metrics']['tokens_out']}")
    print(f"Feedback tokens in: {iteration['feedback_metrics']['tokens_in']}")
    print(f"Feedback tokens out: {iteration['feedback_metrics']['tokens_out']}")
    print(f"Execution time: {iteration['total_iteration_time']:.2f}s")

Total optimization cost: $0.8936
Final score achieved: 0.263

Iteration 0:
Score: 0.263
Comparator tokens in: 27656
Comparator tokens out: 479
Feedback tokens in: 624
Feedback tokens out: 275
Execution time: 90.93s


Now we can evaluate our actor module, for this we've provided an implementation of thread safe evaluator that we above as part of class method of `AvatarOptimizer`.

In [22]:
score, one_shot_metrics = iterative_monkey.thread_safe_evaluator(toolqa_test, optimized_actor_agent)

Processing examples:   0%|          | 0/60 [00:00<?, ?it/s]

The method that achieves the highest F1 score on the CoNLL 2003 English dataset for Named Entity Recognition (NER) is ACE + document-context with an F1 score of 94.6.The method that achieves the highest Mean IoU score on the CamVid dataset for the Semantic Segmentation task is SERNet-Former with a score of 84.62. | PSPNet => 0.0
 | CVT___Multi-Task => 0.0
EffNet-L2 (SAM) achieves the highest Percentage_correct score of 96.08 on the CIFAR-100 dataset for the Image Classification task. | Res2NeXt-29 => 0.0
The method that achieves the highest MAP score on the WikiQA dataset for the Question Answering task is TANDA, which achieved a MAP score of 92%. | Key-Value_Memory_Network => 0.0
The method that achieves the highest accuracy score on the Cora dataset for the node classification task is SSP with an accuracy of 90.16% ± 0.59%. | GCN => 0.0
The method "Discriminative Unsupervised Feature Learning with Convolutional Neural Networks" is evaluated on the STL-10 dataset for the Image Classif

Processing examples:   2%|▏         | 1/60 [02:11<2:09:32, 131.74s/it]

MuZero achieves the highest score on the Atari 2600 Name This Game dataset for the Atari Games task. | IQN => 0.0
The Snips method for Speech Recognition is evaluated on datasets such as the TIMIT Acoustic-Phonetic Continuous Speech Corpus and the Snips SmartLights dataset. These datasets are used to assess various aspects of speech recognition and spoken language understanding. | LibriSpeech_test-clean => 0.0
The method achieving the highest error score on the Yelp Binary classification dataset for Sentiment Analysis is not explicitly mentioned in the available resources. However, the state-of-the-art performance on this dataset is achieved by a shallow-and-wide network with word inputs, which establishes a new state-of-the-art performance with an accuracy of 95.9%. | Char-level_CNN => 0.0
The method achieving the highest Number_of_params score on the WikiText-2 dataset for the Language Modelling task is not explicitly mentioned in the retrieved results. However, the SparseGPT model w

Processing examples:   3%|▎         | 2/60 [03:02<1:21:25, 84.23s/it] 

...

Processing examples:  13%|█▎        | 8/60 [03:15<13:37, 15.71s/it]  

The information about the SVDCNN method achieving the highest error score for the Sentiment Analysis task on a specific dataset is not readily available from the current search results. Further detailed research or access to specific academic papers or datasets might be required to obtain this information. | Yelp_Fine-grained_classification => 0.0


Processing examples: 100%|██████████| 60/60 [03:27<00:00,  3.46s/it]

The Field-gating Seq2seq dual attention method is evaluated on the WikiBio dataset using metrics such as BLEU, ROUGE, and PARENT. | BLEU, ROUGE => 0.5



Evaluation Metrics Report
Execution Time: 209.63 seconds
Total Tokens: 150,298 (1,642 in, 148,656 out)
Total Cost: $1.4907
Average Score: 0.286


In [23]:
print(f"\nIterative evolution summary:")
print(f"Score: {one_shot_metrics['average_score']:.3f}")
print(f"Total Cost: ${one_shot_metrics['total_cost']:.4f}")
print(f"Total Time: {one_shot_metrics['execution_time']:.2f}s")


Iterative evolution summary:
Score: 0.286
Total Cost: $1.4907
Total Time: 209.63s


In [24]:
# iterative_monkey.thread_safe_evaluator(toolqa_test, optimized_actor_agent)
batch_num = 2
score, batch_metrics = iterative_monkey.thread_safe_evaluator_batch(
    toolqa_test, 
    optimized_actor_agent,
    batch_num
)

Processing batch 1 of 2...


Processing examples:   0%|          | 0/60 [00:00<?, ?it/s]

The method that achieves the highest Mean IoU score on the CamVid dataset for the Semantic Segmentation task is SERNet-Former with a score of 84.62. | PSPNet => 0.0
EffNet-L2 (SAM) achieves the highest Percentage_correct score of 96.08 on the CIFAR-100 dataset for the Image Classification task. | Res2NeXt-29 => 0.0
The method that achieves the highest accuracy score on the Cora dataset for the node classification task is SSP with an accuracy of 90.16% ± 0.59%. | GCN => 0.0
The method that achieves the highest F1 score on the CoNLL 2003 English dataset for Named Entity Recognition (NER) is ACE + document-context with an F1 score of 94.6. | CVT___Multi-Task => 0.0
The method "Discriminative Unsupervised Feature Learning with Convolutional Neural Networks" is evaluated on the STL-10 dataset for the Image Classification task. | CIFAR-10 => 0.0
The method "Discriminative Unsupervised Feature Learning with Convolutional Neural Networks" is evaluated on the STL-10 dataset for the Image Classi

Processing examples:   2%|▏         | 1/60 [02:12<2:10:46, 133.00s/it]

MuZero achieves the highest score on the Atari 2600 Name This Game dataset for the Atari Games task. | IQN => 0.0
The iBOWIMG_baseline method achieves the highest Percentage_correct score on the COCO VQA dataset for the Visual Question Answering task. | COCO_Visual_Question_Answering__VQA__real_images_1_0_multiple_choice => 0.5
The Stacked Hourglass Networks method achieves the highest PCK@0.2 score on the FLIC dataset for the Pose Estimation task. | FLIC_Elbows => 0.5
The U-Net method for skin cancer segmentation is commonly evaluated on datasets such as the ISIC 2016, ISIC 2017, and ISIC 2018 datasets. These datasets are part of the International Skin Imaging Collaboration (ISIC) challenges, which provide a large number of dermatoscopic images for training and testing segmentation models. | Kaggle_Skin_Lesion_Segmentation => 0.0
The Field-gating Seq2seq dual attention method is evaluated on the WikiBio dataset using metrics such as BLEU and ROUGE. Additionally, a new metric called PA

...

Processing examples:  33%|███▎      | 20/60 [03:12<02:04,  3.11s/it]

The ACF-WIDER method achieves the highest AP score on the WIDER FACE dataset, with specific AP scores of 0.965 for the easy set, 0.955 for the medium set, and 0.904 for the hard set. | WIDER_Face__Easy_ => 1.0
The current state-of-the-art model for the SNLI dataset with the highest parameters score is not clearly identified in the available resources. The Neural Tree Indexers for Text Understanding is mentioned as a state-of-the-art model, but specific parameter scores are not provided. Further research or direct access to specific academic papers or repositories may be required to obtain this information. | 300D_Residual_stacked_encoders => 0.0


Processing examples: 100%|██████████| 60/60 [03:15<00:00,  3.25s/it]

The Field-gating Seq2seq dual attention method is evaluated on the WikiBio dataset using metrics such as BLEU, ROUGE, and PARENT. | BLEU, ROUGE => 0.5



Batch Evaluation Metrics Report
Total Execution Time: 426.21 seconds
Average Time per Batch: 213.11 seconds
Best Score: 0.303 (Batch 2)
Total Tokens: 285,748 (3,284 in, 282,464 out)
Total Cost: $17.0464

Per-Batch Performance:
--------------------

Batch 1:
  Score: 0.286
  Execution Time: 221.52s
  Tokens: 145,218 (1,642 in, 143,576 out)
  Cost: $8.6638

Batch 2:
  Score: 0.303
  Execution Time: 204.70s
  Tokens: 140,530 (1,642 in, 138,888 out)
  Cost: $8.3825


In [25]:
print(f"\nMixed strategy summary:")
print(f"Best Score: {batch_metrics['final_score']:.3f}")
print(f"Total Cost: ${batch_metrics['total_cost']:.4f}")
print(f"Total Time: {batch_metrics['total_execution_time']:.2f}s")


Mixed strategy summary:
Best Score: 0.303
Total Cost: $17.0464
Total Time: 426.21s
